<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220118_sm_know_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
warnings.filterwarnings(action='ignore') 
know_train = [pd.read_csv(path) for path in sorted(glob('/content/gdrive/MyDrive/know/train/*.csv'))]
know_test = [pd.read_csv(path) for path in sorted(glob('/content/gdrive/MyDrive/know/test/*.csv'))]

In [4]:
train1 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2020.csv', encoding='utf-8')

train3 = train3[train3['idx'] != 19871]

test1 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2017_test.csv', encoding='utf-8')
test2 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2018_test.csv', encoding='utf-8')
test3 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2019_test.csv', encoding='utf-8')
test4 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2020_test.csv', encoding='utf-8')

submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv', encoding='utf-8')

'''
train 2018 : 8551,177,19183,3309(이거는 27이랑 27_1만 바꾸면 됨), 3361('ㅠ21_3 값 숫자로 되어있어야 하는데 한글),7758
test 2019 : 3694, 7090
'''

"\ntrain 2018 : 8551,177,19183,3309(이거는 27이랑 27_1만 바꾸면 됨), 3361('ㅠ21_3 값 숫자로 되어있어야 하는데 한글),7758\ntest 2019 : 3694, 7090\n"

In [5]:
train2 = train2[train2['idx'] != 8551]
train2 = train2[train2['idx'] != 8551]
train2 = train2[train2['idx'] != 8551]
train2 = train2[train2['idx'] != 8551]
train2 = train2[train2['idx'] != 8551]
train2 = train2[train2['idx'] != 8551]

,idx,cq1,cq2,cq3,cq4,cq5,cq6,cq7,cq8,cq9,cq10,cq11,cq12,cq13,cq14,cq15,cq16,cq17,cq18,cq19,cq20,cq21,cq22,cq23,cq24,cq25,cq26,cq27,cq28,cq29,cq30,cq31,cq32,cq33,cq34,cq35,cq36,cq37,cq38,cq39,...,bq233,bq234,bq235,bq241,bq242,bq243,bq244,bq245,bq25,bq25_1,bq26_1,bq26_1a,bq26_2,bq26_2a,bq26_3,bq26_3a,bq26_4,bq26_4a,bq27,bq28,bq28_1,bq29,bq30,bq31,bq32,bq33,bq34,bq35,bq36,bq37,bq37_1,bq38,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode
0,9486,5,3,4,2,2,5,4,5,4,4,4,3,3,4,1,5,3,4.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,4.0,2.0,1.0,1.0,2.0,4.0,3.0,4.0,4.0,3.0,4.0,...,0.0,0,,1,,,,,2,,1,2,1,2,1,1,1,1,2.0,4,정교한 직업 요구의 증가,치기공사,치카공,,교정전문가,,3,1,27,3,치기공과,1,1,,1,42,3800,3400,,306301
1,9487,4,3,4,4,4,4,4,4,4,3,3,3,3,4,3,2,2,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,2.0,2.0,1.0,1.0,3.0,1.0,3.0,3.0,3.0,3.0,...,0.0,0,,,,3,,,1,2,3,,3,,3,,3,,1.0,2,게임 산업의 축소로 인해,없다,"서브라임텍스트, 유니티, 리눅스",정보보호컨설턴트,일반 프로그래머,없다,3,1,28,2,이과,1,2,,1,40,2300,1800,,133204
2,9488,4,1,5,3,3,4,2,4,1,2,1,2,2,2,2,1,1,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,3.0,2.0,4.0,1.0,4.0,3.0,3.0,1.0,4.0,4.0,5.0,4.0,...,0.0,0,5,,,,,5,3,,2,,2,,2,,2,,2.0,2,대형화 되는 추세,사장님,망치 실리콘 사다리 유리칼,회사원,,,4,1,59,2,기계과,2,,6,,30.0,,,3000,702502
3,9489,4,3,4,4,4,3,3,2,3,3,2,3,3,4,4,3,3,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,4.0,3.0,4.0,4.0,2.0,2.0,2.0,2.0,4.0,4.0,3.0,4.0,3.0,3.0,...,0.0,0,,1,,,,,1,2,1,2,1,2,1,2,1,1,2.0,3,"과학이 발달할수록 실험,검사,분석 보조가 필요하므로",,전자현미경 컴퓨터,,,,2,1,38,4,물리 과학,1,1,,1,40,3350,2750,,121201
4,9490,3,2,3,1,1,4,3,4,3,2,3,1,1,5,2,3,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,2.0,2.0,3.0,4.0,4.0,3.0,1.0,1.0,1.0,1.0,...,0.0,0,,1,,,,,1,1,1,2,1,2,1,2,1,1,2.0,4,고령화로인하여 틀니 및 인플란트등 의료보험 보장범위가 확대되어서,,"니켈, 크롬, 코발트",판매업,,,4,2,27,4,치기공학,1,1,,1,40,2500,2300,,306301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9067,18564,5,5,5,2,2,4,2,5,3,5,4,1,4,5,3,1,1,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,2.0,2.0,3.0,2.0,3.0,1.0,3.0,4.0,3.0,5.0,...,0.0,0,5,,,,,5,1,1,2,,2,,2,,2,,2.0,1,한약시장의 불투명성과 신뢰도하락. 미디어에 의해 대체되고 건강기능식품에 자리를 내줄...,없음,전탕기.포장기,없음,없음,없음,5,1,26,4,한약학,2,,6,,60,,,2000,303002
9068,18565,5,5,5,3,3,5,4,4,4,5,5,1,4,5,2,2,4,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,3.0,2.0,2.0,5.0,...,0.0,0,5,,,,,5,1,1,1,1,2,,2,,2,,2.0,2,기계화되어 일력이 덜 필요할 것 같다,,컴퓨터,현직과 동일,통신기사,없다,3,1,56,4,국문과,2,,6,,52.0,,,5000,615203
9069,18566,4,4,4,3,3,5,4,4,4,3,4,4,4,5,1,1,3,3.0,1.0,1.0,4.0,4.0,1.0,5.0,1.0,1.0,1.0,1.0,4.0,2.0,2.0,4.0,2.0,5.0,1.0,4.0,4.0,5.0,5.0,...,0.0,0.0,5,,,,,5,1,1,2,,2,,2.0,,2.0,,2.0,2,의료직업 인구수가 많아져서,비뇨기과 전문의,의료기구,없음,없음,없음,3,2,66,6,의대,2,,5,,38,,,9000,301106
9070,18567,5,2,3,2,3,5,4,4,3,3,4,3,2,5,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,5.0,1.0,2.0,4.0,3.0,1.0,3.0,3.0,5.0,5.0,3.0,4.0,5.0,4.0,...,0.0,0,5,,,,,5,1,2,1,1,1,1,1,1,1,1,1.0,3,지금과 비슷,피부과의사,각종 레이저기기,없다,없다,피부과의사,3,1,45,4,의예,2,,5,,56,,,6500,301112


In [ ]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [ ]:
for df in know_test:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [ ]:
years = ['2017', '2018', '2019', '2020']

rf_predicts = []
year_encoder = {}

for year, train, test in zip(years, know_train, know_test):
    print(year)
    encoders = {}
    for col in train.columns:
        if col == 'ID':
            continue
        try:
            train[col] = train[col].map(int)
        except:
            encoder = LabelEncoder()
            train[col] = train[col].map(str)
            train[col] = encoder.fit_transform(train[col])
            encoders[col] = encoder  
    year_encoder[year] = encoders

    feature = train.columns.drop(['idx', 'knowcode'])

    model = RandomForestClassifier(n_estimators=100, random_state=123456, n_jobs=8)
    model.fit(train[feature], train['knowcode'])

    for col in test.columns:
      try:
          test[col] = test[col].map(int)
      except:
          encoder = year_encoder[year][col]
          test[col] = test[col].map(str)
          category_map = {category: idx for idx, category in enumerate(encoder.classes_)}
          test[col] = test[col].apply(lambda x: category_map[x] if x in category_map else -1) # train set에서 보지못한 카테고리변수 -1(UNK) 처리
    feature = test.columns.drop(['idx'])
    pred = model.predict(test[feature])
    rf_predicts.extend(pred)

2017
2018
2019
2020


In [ ]:
submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv') # sample submission 불러오기

In [ ]:
submission['knowcode'] = rf_predicts

In [ ]:
submission

,idx,knowcode
0,0,29401
1,1,140306
2,2,122102
3,3,121102
4,4,412003
...,...,...
35226,35244,833001
35227,35245,29902
35228,35246,833001
35229,35247,154101


In [ ]:
submission.to_csv('/content/gdrive/MyDrive/know/submission.csv', index=False)